## Кривоногов Н.В., NLP, практическое задание № 12 - 1

In [1]:
!pip install -q youtokentome
!pip install -q datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.7/86.7 kB 1.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.6/519.6 kB 5.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 12.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 20.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 13.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.8/294.8 kB 26.9 MB/s eta 0:00:00


In [2]:
from datasets import load_dataset
from youtokentome import BPE

In [3]:
dataset = load_dataset("ag_news")

Generating train split:   0%|          | 0/120000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/7600 [00:00<?, ? examples/s]

In [4]:
with open("data.txt", "w") as out:
    for data_d in dataset["train"]:
        out.write(data_d["text"]+"\n")

In [5]:
data_d['text']

'Nets get Carter from Raptors INDIANAPOLIS -- All-Star Vince Carter was traded by the Toronto Raptors to the New Jersey Nets for Alonzo Mourning, Eric Williams, Aaron Williams, and a pair of first-round draft picks yesterday.'

In [6]:
bpe = BPE.train(data="data.txt", vocab_size=5000, model="bpe.bin")

In [7]:
encoded = bpe.encode("Hello there! What is BPE?")

In [8]:
tokenized = ""
for word in encoded:
    tokenized += " " + bpe.id_to_subword(word)
tokenized

' ▁H ell o ▁there ! ▁What ▁is ▁B P E ?'

## GPT 2


## Huggingface's transformers

![](https://raw.githubusercontent.com/huggingface/transformers/master/docs/source/imgs/transformers_logo_name.png)

In [9]:
!pip install -q transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 63.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 114.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 55.5 MB/s eta 0:00:00


In [10]:
from transformers import GPT2LMHeadModel
from transformers import GPT2Tokenizer

In [11]:
#sberbank-ai/rugpt3medium_based_on_gpt2

tokenizer = GPT2Tokenizer.from_pretrained("sberbank-ai/rugpt3medium_based_on_gpt2")
ru_gpt = GPT2LMHeadModel.from_pretrained("sberbank-ai/rugpt3medium_based_on_gpt2", output_attentions=True)

In [12]:
device = "cuda"
ru_gpt.to(device)

GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(50257, 1024)
    (wpe): Embedding(2048, 1024)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0-23): 24 x GPT2Block(
        (ln_1): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D()
          (c_proj): Conv1D()
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
  )
  (lm_head): Linear(in_features=1024, out_features=50257, bias=False)
)

In [14]:
inp = "10 причин для окончания специальной военной операции в Украине"

encoded = tokenizer.encode(inp, return_tensors="pt")


result = ru_gpt.generate(
    encoded.to(device),
    max_length=1000,
    early_stopping=True,
    no_repeat_ngram_size=3)

print(tokenizer.decode(result.cpu().flatten().numpy(), skip_special_tokens=True))

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


10 причин для окончания специальной военной операции в Украине.  Об этом заявил в эфире телеканала "112 Украина" заместитель главы Администрации Президента Украины Валерий Чалый. 
 "Мы не можем сейчас сказать, что мы не можем закончить специальную операцию в Украине, потому что мы должны понимать, что это не просто война, это война с Россией.  И мы должны четко понимать, кто и как будет воевать.  Мы должны понимать и то, что если мы не будем воевать, то мы не сможем победить", - сказал он. 
 По словам Чалого, в настоящее время в Украине идет подготовка к проведению специальной операции. 
 Напомним, ранее сообщалось, что в Украине началась спецоперация по задержанию российских диверсантов. 

 В Москве в рамках проекта "Культурная столица" пройдет выставка "Память.  История.  Современность" 
 В Москве с 24 по 27 сентября в рамках программы "Культура.  События" пройдет международная выставка "История.  Время.  Память. 2015".  В экспозиции будут представлены работы художников, скульпторов,